In [1]:
from Acesso import Login
from Query import Query
import pandas as pd

pd.set_option('display.max_columns',None)

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)



In [2]:
querys={
    
    'Rota':
    
    """
    
    SELECT * FROM netfeira.vw_roteiros
    WHERE [Data da Montagem] BETWEEN (SELECT MIN(Data) 
    FROM netfeira.vw_calend
    WHERE YEAR(Data)=YEAR(GETDATE()) AND MONTH(Data)=MONTH(GETDATE())) AND
    (SELECT MAX(Data) 
    FROM netfeira.vw_calend
    WHERE YEAR(Data)=YEAR(GETDATE()) AND MONTH(Data)=MONTH(GETDATE()))
    ORDER BY [Data da Montagem] 
    
    """,
    
    'Calendario':
    
    """
    
    SELECT * FROM netfeira.vw_calend
    WHERE YEAR(Data)=YEAR(GETDATE()) AND MONTH(Data)=MONTH(GETDATE())    
    
    """,
    
    'Log':
    
    """
    
    SELECT * FROM netfeira.vw_log_conta
    
    """,
    
    'Meta':
    
    """
    
    SELECT * FROM netfeira.vw_metas    
    
    """
    
}

In [3]:
df=sql.CriarTabela(kwargs=querys)

In [4]:
meta=df['Meta']['Meta R$'].sum()

meta

4201623.14

In [5]:
perc=df['Log']['Perc'].values[-1]

In [6]:
frete=df['Rota']['Frete R$'].sum()

In [7]:
util=df['Calendario'].loc[df['Calendario']['Dia Útil']==1,'Data'].count()

meta_frete=round(meta*perc,2)

meta_dia=round(meta_frete/util,2)

In [8]:
df['Semana']=df['Calendario'].loc[df['Calendario']['Dia Útil']==1].groupby(['Semana Ano'],as_index=False).agg({'Dia Útil':'count'})

In [9]:
df['Semana']['Meta Semanal']=df['Semana']['Dia Útil']*meta_dia

In [10]:
df['Semana']['Data Mín']=df['Semana']['Semana Ano'].apply(lambda info: df['Calendario']['Data'].loc[df['Calendario']['Semana Ano']==info].min())

df['Semana']['Data Máx']=df['Semana']['Semana Ano'].apply(lambda info: df['Calendario']['Data'].loc[df['Calendario']['Semana Ano']==info].max())

In [11]:
df['Semana']['Frete R$']=df['Semana'].apply(lambda info: df['Rota']['Frete R$'].loc[df['Rota']['Data da Montagem'].between(info['Data Mín'],info['Data Máx'])].sum(),axis=1)

In [12]:
df['Semana']['Dif']=round(df['Semana']['Meta Semanal']-df['Semana']['Frete R$'],2)

In [13]:
soma=0

valores=[]

for i,total in enumerate(df['Semana']['Dif']):
            
    if(i==0):
        
        valores.append(0)
        
        continue
        
    dif=df['Semana']['Dif'].iloc[i-1].sum()
    
    valores.append(dif)
    
    pass

df['Semana']['Ant Dif']=valores

In [14]:
df['Semana']['Meta Semanal']=round(df['Semana']['Meta Semanal']+df['Semana']['Ant Dif'],2)

In [15]:
df['Semana']['Dif']=round(df['Semana']['Meta Semanal']-df['Semana']['Frete R$'],2)

In [16]:
df['Semana'].drop(columns=['Ant Dif'],inplace=True)

In [20]:
df['Semana']['Perc']=round(df['Semana']['Frete R$']/df['Semana']['Meta Semanal'],2)*100

In [22]:
df['Semana'].loc[df['Semana']['Frete R$']<=0,'Dif']=0

In [23]:
df['Semana']

,Semana Ano,Dia Útil,Meta Semanal,Data Mín,Data Máx,Frete R$,Dif,Perc
0,49,2,15278.64,2022-12-01,2022-12-03,16340.0,-1061.36,107.0
1,50,5,37135.24,2022-12-04,2022-12-10,17570.0,19565.24,47.0
2,51,5,58823.20,2022-12-11,2022-12-17,0.0,0.00,0.0
3,52,5,76393.20,2022-12-18,2022-12-24,0.0,0.00,0.0
4,53,5,76393.20,2022-12-25,2022-12-31,0.0,0.00,0.0


In [28]:
semana_ano=df['Semana'].loc[df['Semana']['Frete R$']>0,'Semana Ano'].max()

In [30]:
df['Semana'].loc[df['Semana']['Semana Ano']==semana_ano,'Meta Semanal'].sum()

37135.24

In [33]:
round(frete/meta_frete,2)*100

20.0

In [37]:
df['Situacao']=df['Rota'].groupby(['Situação da Rota'],as_index=False).agg({'Frete R$':'sum'})

In [39]:
df['Rota'].columns

Index(['Empresa', 'Romaneio', 'Seq. Roteiro', 'Rota', 'Pedido', 'NFe',
       'Situação da Rota', 'Data da Montagem', 'Data da Saída',
       'Data de Retorno', 'Data da Separação', 'ID Motor', 'Motorista',
       'ID Veículo', 'Veículo', 'Frete Pago', 'Tipo de Rota', 'ID Cliente',
       'ID Vendedor', 'Data do Pedido', 'Data de Faturamento', 'SKU', 'Seq.',
       'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Venda',
       'Peso Bruto KG', 'Peso Líquido KG', 'Frete R$'],
      dtype='object')

In [41]:
df['Situacao']['Pedidos']=df['Situacao']['Situação da Rota'].apply(lambda info: len(df['Rota']['Pedido'].loc[df['Rota']['Situação da Rota']==info].unique().tolist()))

In [43]:
df['Situacao'].sort_values('Pedidos',ascending=False,inplace=True)

In [46]:
df['Situacao']['Perc']=round(df['Situacao']['Pedidos']/len(df['Rota']['Pedido'].unique().tolist()),4)*100

In [49]:
df['Situacao']['Perc'].loc[df['Situacao']['Situação da Rota']=='EM ABERTO'].max()

34.72

In [51]:
df['Regiao']=df['Rota'].groupby(['Rota'],as_index=False).agg({'Frete R$':'sum'})

In [53]:
df['Regiao']['Pedidos']=df['Regiao']['Rota'].apply(lambda info: len(df['Rota']['Pedido'].loc[df['Rota']['Rota']==info].unique().tolist()))

In [55]:
df['Regiao'].sort_values('Pedidos',ascending=False,inplace=True)

In [57]:
df['Regiao']['Clientes']=df['Regiao']['Rota'].apply(lambda info: len(df['Rota']['ID Cliente'].loc[df['Rota']['Rota']==info].unique().tolist()))

In [59]:
df['Regiao'].columns

Index(['Rota', 'Frete R$', 'Pedidos', 'Clientes'], dtype='object')